#### Set the input parameters

In [1]:
center_lat = 28.587784 # latitude of center point, in decimal degrees for example 28.587784, 77.238539
center_lon = 77.238539 # longitude of center point, in decimal degrees
radius = 5 # radius of flower of life, in kilometres
rotation = 0 # optional: add a tilt to the grid. I do not recommend tilting your grids
max_points = 300 # the number of points you want the grid to be

#### Logic for calculating distances between points and creating pattern

In [ ]:
divisions = 6
round_to_decimal_place = 2
max_points -= 1

In [ ]:
# from math import asin, atan2, cos, degrees, radians, sin
# import numpy as np
# import pandas as pd
# from google.colab import files
# import math

# # R = 6371

# def get_point_at_distance(lat1, lon1, d, bearing, R=6371.0088):
#     """
#     lat: initial latitude, in degrees
#     lon: initial longitude, in degrees
#     d: target distance from initial
#     bearing: (true) heading in degrees
#     R: optional radius of sphere, defaults to mean radius of earth

#     Returns new lat/lon coordinate {d}km from initial, in degrees
#     """
#     lat1 = radians(lat1)
#     lon1 = radians(lon1)
#     a = radians(bearing)
#     lat2 = asin(sin(lat1) * cos(d/R) + cos(lat1) * sin(d/R) * cos(a))
#     lon2 = lon1 + atan2(
#         sin(a) * sin(d/R) * cos(lat1),
#         cos(d/R) - sin(lat1) * sin(lat2)
#     )
#     return (degrees(lat2), degrees(lon2),)



In [ ]:
import geopy
from geopy.distance import geodesic
from math import asin, atan2, cos, degrees, radians, sin
from geopy import Point
import pandas as pd
from google.colab import files

def get_point_at_distance(lat1, lon1, d, deg_bearing):

  # given: lat1, lon1, b = bearing in degrees, d = distance in kilometers

  #origin = geopy.Point(lat1, lon1)
  origin = (lat1, lon1)
  # destination = geodesic(kilometers=d).destination(origin, bearing)
  lat2,lon2 = geodesic(kilometers=d).destination(origin, bearing=deg_bearing).format_decimal().split(',')
  #lat2,lon2 = geopy.distance(kilometers=d).destination((lat1, lon1), bearing)
  #lat2, lon2 = destination.latitude, destination.longitude
  return (float(lat2), float(lon2),)


In [ ]:
# up - full vertical
# top right - full vertical
# top left - full vertical
# top right from top right - full vertical
# top left from top left - full vertical
# top right from top right from top right - full vertical
# top left from top left from top left - full vertical

divisions = 6 # creates flower of life by dividing the 360 degree circle into 6 equal parts

cuts = []

tmp_cut = 1

for i in range(1, 2^1000000):
  tmp_cut += divisions*i
  cuts.append(tmp_cut)
  if tmp_cut > max_points:
    break

results_df = pd.DataFrame({'lat':[center_lat],
                              'lon':[center_lon]})

max_iteration_number = len(cuts)

right_start_points = pd.DataFrame({'lat':[],
                              'lon':[]})

left_start_points = pd.DataFrame({'lat':[],
                              'lon':[]})

# up from center coord

for j in range(1, 2*max_iteration_number+1):
      start_point = results_df.loc[0]

      tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'],
                                                           start_point['lon'], radius*j, 0+rotation)

      df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

      results_df = pd.concat([results_df, df_tmp_coord])
      results_df = results_df.reset_index(drop=True)

# down from center coord

for j in range(1, 2*max_iteration_number+1):
      start_point = results_df.loc[0]

      tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'],
                                                           start_point['lon'], radius*j, 180+rotation)

      df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

      results_df = pd.concat([results_df, df_tmp_coord])
      results_df = results_df.reset_index(drop=True)


# top right start pt

start_point = results_df.loc[0]

tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'],
                                                        start_point['lon'], radius, 60+rotation)

df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

right_start_points = pd.concat([right_start_points, df_tmp_coord])
right_start_points = right_start_points.reset_index(drop=True)

results_df = pd.concat([results_df, df_tmp_coord])
results_df = results_df.reset_index(drop=True)

# top left start pt

start_point = results_df.loc[0]

tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'],
                                                        start_point['lon'], radius, 300+rotation)

df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

left_start_points = pd.concat([left_start_points, df_tmp_coord])
left_start_points = left_start_points.reset_index(drop=True)

results_df = pd.concat([results_df, df_tmp_coord])
results_df = results_df.reset_index(drop=True)

for iteration_number in range(1, max_iteration_number+1):

    # next top right start pt

    start_point = right_start_points.loc[iteration_number-1]

    tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'],
                                                            start_point['lon'], radius, 60+rotation)

    df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

    right_start_points = pd.concat([right_start_points, df_tmp_coord])
    right_start_points = right_start_points.reset_index(drop=True)

    results_df = pd.concat([results_df, df_tmp_coord])
    results_df = results_df.reset_index(drop=True)


    for j in range(1, 2*max_iteration_number+1):
      # go up from top right coord

      start_point = right_start_points.loc[iteration_number-1]

      tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'],
                                                           start_point['lon'], radius*j, 0+rotation)

      df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

      results_df = pd.concat([results_df, df_tmp_coord])
      results_df = results_df.reset_index(drop=True)

    for j in range(1, 2*max_iteration_number+1):
      # go down from top right coord

      start_point = right_start_points.loc[iteration_number-1]

      tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'],
                                                           start_point['lon'], radius*j, 180+rotation)

      df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

      results_df = pd.concat([results_df, df_tmp_coord])
      results_df = results_df.reset_index(drop=True)

    # next top left start pt

    start_point = left_start_points.loc[iteration_number-1]

    tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'],
                                                            start_point['lon'], radius, 300+rotation)

    df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

    left_start_points = pd.concat([left_start_points, df_tmp_coord])
    left_start_points = left_start_points.reset_index(drop=True)


    results_df = pd.concat([results_df, df_tmp_coord])
    results_df = results_df.reset_index(drop=True)

    for j in range(1, 2*max_iteration_number+1):
      # go up from top left coord

      start_point = left_start_points.loc[iteration_number-1]

      tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'],
                                                           start_point['lon'], radius*j, 0+rotation)

      df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

      results_df = pd.concat([results_df, df_tmp_coord])
      results_df = results_df.reset_index(drop=True)

    for j in range(1, 2*max_iteration_number+1):
      # go down from top left coord

      start_point = left_start_points.loc[iteration_number-1]

      tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'],
                                                           start_point['lon'], radius*j, 180+rotation)

      df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

      results_df = pd.concat([results_df, df_tmp_coord])
      results_df = results_df.reset_index(drop=True)


start_point = results_df.loc[0]
distances_of_points_from_center = []

for i, r in results_df.iterrows():
  tmp_distance = geodesic(start_point, (r['lat'],r['lon']))
  distances_of_points_from_center.append(tmp_distance)

results_df2 = results_df
results_df2['distance'] = distances_of_points_from_center

keep_till_this_iteration = (max_iteration_number-1)*radius

results_df2 = results_df2.sort_values(by='distance')
results_df2 = results_df2.reset_index(drop=True)

results_df3 = results_df2[results_df2['distance']<=keep_till_this_iteration*1.03]

df_remaining = results_df2[results_df2['distance']>keep_till_this_iteration*1.03]
df_remaining = df_remaining.reset_index(drop=True)

keep_these_remaining = df_remaining.loc[:max_points-len(results_df3)]

results_df4 = pd.concat([results_df3, keep_these_remaining])

results_df4 = results_df4[['lat','lon']]
results_df4 = results_df4.reset_index(drop=True)

#### Creation of Output file. Import it into Google Earth by doing File -> Import and clicking on output.txt

In [ ]:
 # pastes the resulting coordinates into a text file to import into Google Earth
output_text = ''
output_text += 'latitude'+','+'longitude'
output_text += '\n'
for i,r in results_df4.iterrows():
  tmp_pt = str(r['lat'])+','+str(r['lon'])+'\n'
  output_text += tmp_pt

f = open("output.txt", "w")
f.write(output_text)
files.download('output.txt')
f.close()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>